### Import Necessary Libaries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes # Uncomment out if you need to install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes # Uncomment out you need to install folium
import folium # plotting library

!pip install xlrd


print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

After importing/installing all the necessary packages, we will read the excel spreadsheet. The neighborhood and location columns are from wikipedia. I added the state and combo column to allow geolocator to work in the following code.

In [2]:
# Reading table into dataframe
bmore_df = pd.read_excel('Baltimore_Neigh_List.xlsx')
print(bmore_df.shape)
bmore_df

(221, 5)


,Neighborhood,City,Location,State,Combo
0,Abell,Baltimore,North,MD,"Abell, Baltimore, MD"
1,Allendale,Baltimore,West,MD,"Allendale, Baltimore, MD"
2,Arcadia,Baltimore,Northeast,MD,"Arcadia, Baltimore, MD"
3,Arlington,Baltimore,Northwest,MD,"Arlington, Baltimore, MD"
4,Armistead Gardens,Baltimore,Northeast,MD,"Armistead Gardens, Baltimore, MD"
...,...,...,...,...,...
216,Woodring,Baltimore,Northeast,MD,"Woodring, Baltimore, MD"
217,Wyman Park,Baltimore,North,MD,"Wyman Park, Baltimore, MD"
218,Wyndhurst,Baltimore,North,MD,"Wyndhurst, Baltimore, MD"
219,Yale Heights,Baltimore,Southwest,MD,"Yale Heights, Baltimore, MD"


Here we are reading the data in the combo column and using a for-loop

In [3]:
# allows to read the specific column of the spreadsheet
def readcolumn(filename,column):
    #select sheet name and selct column as index,index_col=0
    df = pd.read_excel(filename)
    headername = list(df)
    print(headername)
    column_data =df[list(df)[column]].tolist()
    return  column_data

neigh_list = readcolumn('Baltimore_Neigh_List.xlsx', 4)

['Neighborhood', 'City', 'Location', 'State', 'Combo']


In [4]:
# Prints out the neighborhood list with the latitude and longitude
df = []
for neigh in neigh_list:
    try:
        geolocator = Nominatim(user_agent="bmore_explorer")
        location = geolocator.geocode(neigh)
        latitude = location.latitude
        longitude = location.longitude
        print(neigh, latitude, longitude)
        df.append({'Location':location,
                  'Latitude': latitude,
                  'Longitude': longitude})
    except Exception as e:
        print('Error, skipping address...', e)

Abell, Baltimore, MD 39.3261388 -76.611737
Allendale, Baltimore, MD 39.2916866 -76.6769424
Arcadia, Baltimore, MD 39.3340564 -76.5664996
Arlington, Baltimore, MD 39.3486919 -76.6826661
Armistead Gardens, Baltimore, MD 39.3066459 -76.5562793455641
Arundel Cove, Baltimore, MD 39.1985156 -76.5673626074159
Ashburton, Baltimore, MD 39.3279621 -76.6710811
Baltimore Highlands, Baltimore, MD 39.2331629 -76.636634
Barclay, Baltimore, MD 39.3149545 -76.6122023
Barre Circle, Baltimore, MD 39.2848699 -76.6276523
Bayview, Baltimore, MD 39.2894325 -76.5416232
Beechfield, Baltimore, MD 39.2752308 -76.6975606
Belair-Edison, Baltimore, MD 39.3215607 -76.5678685
Belair-Parkside, Baltimore, MD 39.327438 -76.566113
Berea, Baltimore, MD 39.3092351 -76.5779746
Beverly Hills, Baltimore, MD 39.3402861 -76.5694739
Biddle Street, Baltimore, MD 39.3093234 -76.5260695
Bolton Hill, Baltimore, MD 39.3084715 -76.6259787
Brewer's Hill, Baltimore, MD 39.2838792 -76.5645376
Broadway East, Baltimore, MD 39.308818 -76.58

In [5]:
# Convert list to dataframe
df = pd.DataFrame.from_dict (df)
df

,Location,Latitude,Longitude
0,"(Abell, Baltimore, Maryland, 21218, USA, (39.3...",39.326139,-76.611737
1,"(Allendale, Baltimore, Maryland, 21229, USA, (...",39.291687,-76.676942
2,"(Arcadia, Baltimore, Maryland, 21082:21234, US...",39.334056,-76.566500
3,"(Arlington, Baltimore, Maryland, 21215, USA, (...",39.348692,-76.682666
4,"(Armistead Gardens, Baltimore, Maryland, USA, ...",39.306646,-76.556279
...,...,...,...
216,"(Woodring, Baltimore, Maryland, 21256, USA, (3...",39.363819,-76.544286
217,"(Wyman Park, Baltimore, Maryland, 21211, USA, ...",39.333540,-76.626371
218,"(Wyndhurst, Baltimore, Maryland, 21210, USA, (...",39.355871,-76.631357
219,"(Yale Heights, Baltimore, Maryland, 21229, USA...",39.275706,-76.689895


In [6]:
# Merging the two dataframes
df_merged = df.merge(bmore_df, how='outer', left_index=True, right_index=True)
df_merged

,Location_x,Latitude,Longitude,Neighborhood,City,Location_y,State,Combo
0,"(Abell, Baltimore, Maryland, 21218, USA, (39.3...",39.326139,-76.611737,Abell,Baltimore,North,MD,"Abell, Baltimore, MD"
1,"(Allendale, Baltimore, Maryland, 21229, USA, (...",39.291687,-76.676942,Allendale,Baltimore,West,MD,"Allendale, Baltimore, MD"
2,"(Arcadia, Baltimore, Maryland, 21082:21234, US...",39.334056,-76.566500,Arcadia,Baltimore,Northeast,MD,"Arcadia, Baltimore, MD"
3,"(Arlington, Baltimore, Maryland, 21215, USA, (...",39.348692,-76.682666,Arlington,Baltimore,Northwest,MD,"Arlington, Baltimore, MD"
4,"(Armistead Gardens, Baltimore, Maryland, USA, ...",39.306646,-76.556279,Armistead Gardens,Baltimore,Northeast,MD,"Armistead Gardens, Baltimore, MD"
...,...,...,...,...,...,...,...,...
216,"(Woodring, Baltimore, Maryland, 21256, USA, (3...",39.363819,-76.544286,Woodring,Baltimore,Northeast,MD,"Woodring, Baltimore, MD"
217,"(Wyman Park, Baltimore, Maryland, 21211, USA, ...",39.333540,-76.626371,Wyman Park,Baltimore,North,MD,"Wyman Park, Baltimore, MD"
218,"(Wyndhurst, Baltimore, Maryland, 21210, USA, (...",39.355871,-76.631357,Wyndhurst,Baltimore,North,MD,"Wyndhurst, Baltimore, MD"
219,"(Yale Heights, Baltimore, Maryland, 21229, USA...",39.275706,-76.689895,Yale Heights,Baltimore,Southwest,MD,"Yale Heights, Baltimore, MD"


In [7]:
# Removing the whitespace and renaming
df_merged.columns = ['location', 'Latitude', 'Longitude', 'Neighborhood', 'City', 'Location', 'State', 'Combo', ]
print(df_merged.columns.tolist())

['location', 'Latitude', 'Longitude', 'Neighborhood', 'City', 'Location', 'State', 'Combo']


In [8]:
# Creating a new dataframe with the necessary columns
df_final = df_merged[['Neighborhood', 'Location', 'Latitude', 'Longitude']]
df_final

,Neighborhood,Location,Latitude,Longitude
0,Abell,North,39.326139,-76.611737
1,Allendale,West,39.291687,-76.676942
2,Arcadia,Northeast,39.334056,-76.566500
3,Arlington,Northwest,39.348692,-76.682666
4,Armistead Gardens,Northeast,39.306646,-76.556279
...,...,...,...,...
216,Woodring,Northeast,39.363819,-76.544286
217,Wyman Park,North,39.333540,-76.626371
218,Wyndhurst,North,39.355871,-76.631357
219,Yale Heights,Southwest,39.275706,-76.689895


In [9]:
# Ensure the types are correct for mapping
print(df_final.dtypes)

Neighborhood     object
Location         object
Latitude        float64
Longitude       float64
dtype: object


We will now see if any data was not merged correctly or if any is not present

In [10]:
# Calculate how many rows are missing
missing_data = df_final.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

Neighborhood
False    221
Name: Neighborhood, dtype: int64

Location
False    221
Name: Location, dtype: int64

Latitude
False    221
Name: Latitude, dtype: int64

Longitude
False    221
Name: Longitude, dtype: int64



It looks like all of the data was transferred correctly. 221 out of 221 rows!

# -------------------------------------------------------------------------------------

## Explore and cluster the neighborhoods in Baltimore

In [11]:
# create map of Baltimore using latitude and longitude values from above
map_baltimore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map from 'can_loc' df
for lat, lng, location, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Location'], df_final['Neighborhood']):
    label = '{}, {}'.format(location, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_baltimore)  
    
map_baltimore

In [12]:
# View Neighborhood's total by cardinal direction
pd.value_counts(df_final['Location'].values, sort=False)

Northeast    31
Southeast    23
West         18
Southwest    26
Central      18
Northwest    27
North        44
East         16
South        18
dtype: int64

### Define credentials

In [13]:
CLIENT_ID = 'UQA1MEJBGS4DSL2QU43GDK3FE3CC5121QVWZUWZUCYPOGWAS' # your Foursquare ID
CLIENT_SECRET = 'MFSQLKINWED3HSACNIR44SHO3QKF5BH42ISYT1VFJ5LKSYUX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + 'ENTERED')
print('CLIENT_SECRET:' + 'ENTERED')

Your credentails:
CLIENT_ID: ENTERED
CLIENT_SECRET:ENTERED


In [14]:
df_final.loc[0]

Neighborhood      Abell
Location          North
Latitude        39.3261
Longitude      -76.6117
Name: 0, dtype: object

Finding the top 100 venues around Baltimore

In [15]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print('URL entered with credentials')

URL entered with credentials


In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6db06df9dbde003860a94c'},
 'response': {'venues': [{'id': '4bdc5ae12a3a0f475372b2b6',
    'name': 'APlus at Sunoco',
    'location': {'address': '4719 York Rd',
     'crossStreet': 'Cold Spring and York',
     'lat': 39.34647007275735,
     'lng': -76.60939036042217,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.34647007275735,
       'lng': -76.60939036042217}],
     'distance': 38,
     'postalCode': '21212',
     'cc': 'US',
     'city': 'Baltimore',
     'state': 'MD',
     'country': 'United States',
     'formattedAddress': ['4719 York Rd (Cold Spring and York)',
      'Baltimore, MD 21212',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-156746

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Since we want to find the popular venues by location, we are going to first analyze the data through geographical location and will later analyze each neighborhood.

In [21]:
baltimore_venues = getNearbyVenues(names=df_final['Location'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

North
West
Northeast
Northwest
Northeast
South
Northwest
Southeast
East
Southwest
Southeast
Southwest
Northeast
Northeast
East
Northeast
East
Central
Southeast
East
South
South
Southeast
Northwest
North
Southeast
Southwest
Central
North
Northeast
Northeast
Northwest
Central
North
North
South
Northwest
Northeast
North
West
Northwest
South
North
East
Southwest
Northwest
Northwest
Central
East
Northwest
East
West
Southeast
Southwest
West
Northeast
East
North
Southwest
Northwest
South
Southeast
Southeast
Northwest
Northeast
West
Southwest
Northeast
East
Northwest
North
Northeast
North
Southeast
Southeast
East
North
Northwest
North
Northwest
North
Northwest
Southeast
Southeast
South
West
North
South
West
Southeast
Northeast
Southwest
North
Northeast
Southwest
North
Northwest
Northeast
Central
Southwest
East
Southeast
Southeast
North
North
North
Southeast
North
North
South
Northwest
Northeast
North
West
Central
Northwest
Southeast
South
North
Northwest
Central
Central
South
Northeast
East
No

### Examine the new dataframe

Now lets view the new dataframe and shape

In [22]:
print(baltimore_venues.shape)
baltimore_venues.head()

(3867, 7)


,Location,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North,39.326139,-76.611737,32nd Street Farmer's Market,39.327362,-76.610851,Farmers Market
1,North,39.326139,-76.611737,Pete's Grille,39.327064,-76.609593,Breakfast Spot
2,North,39.326139,-76.611737,The Book Thing,39.325253,-76.610272,Bookstore
3,North,39.326139,-76.611737,Normal's Books & Records,39.326012,-76.609903,Record Shop
4,North,39.326139,-76.611737,Peabody Heights Brewery,39.324744,-76.610819,Brewery


In [23]:
# one hot encoding
baltimore_onehot = pd.get_dummies(baltimore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
baltimore_onehot['Location'] = baltimore_venues['Location']

In [24]:
# move neighborhood column to the first column
neighborhood = baltimore_onehot['Location']
baltimore_onehot.drop(labels=['Location'], axis=1,inplace = True)
baltimore_onehot.insert(0, 'Location', neighborhood)
baltimore_onehot

,Location,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3862,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3863,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3864,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3865,Northeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
baltimore_onehot.shape

(3867, 287)

Adding the venues expanded the dataframe from (3872, 7) to (3872, 291)

We will now group the neighborhoods by taking the mean of the frequency of occurrence of each category

In [26]:
baltimore_grouped = baltimore_onehot.groupby('Location').mean().reset_index()
baltimore_grouped

,Location,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central,0.000000,0.000000,0.007926,0.002642,0.00000,0.027741,0.000000,0.001321,0.010568,...,0.007926,0.000000,0.000000,0.000000,0.00000,0.007926,0.000000,0.000000,0.000000,0.001321
1,East,0.000000,0.000000,0.000000,0.000000,0.00000,0.051136,0.000000,0.000000,0.000000,...,0.005682,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.017045,0.000000,0.000000
2,North,0.005236,0.001047,0.001047,0.002094,0.00000,0.034555,0.000000,0.003141,0.000000,...,0.002094,0.000000,0.001047,0.001047,0.00000,0.004188,0.004188,0.000000,0.001047,0.008377
3,Northeast,0.004184,0.000000,0.000000,0.000000,0.00000,0.037657,0.004184,0.000000,0.016736,...,0.000000,0.000000,0.000000,0.004184,0.00000,0.000000,0.004184,0.000000,0.008368,0.000000
4,Northwest,0.000000,0.000000,0.000000,0.000000,0.00000,0.048544,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.029126,0.000000,0.000000
5,South,0.000000,0.000000,0.000000,0.000000,0.00000,0.011905,0.000000,0.000000,0.000000,...,0.000000,0.005952,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.008929
6,Southeast,0.001230,0.000000,0.001230,0.000000,0.00123,0.034440,0.000000,0.000000,0.000000,...,0.002460,0.000000,0.000000,0.000000,0.00492,0.002460,0.003690,0.000000,0.008610,0.008610
7,Southwest,0.000000,0.000000,0.003257,0.003257,0.00000,0.022801,0.000000,0.000000,0.000000,...,0.006515,0.000000,0.003257,0.000000,0.00000,0.003257,0.000000,0.013029,0.000000,0.000000
8,West,0.011050,0.000000,0.005525,0.005525,0.00000,0.049724,0.000000,0.000000,0.000000,...,0.011050,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.022099,0.000000,0.000000


Grouping the neighborhoods identified 9 unique values, decreasing the dataframe to 9 rows and it also automatically sorted the neighborhoods

Let's identify the top 10 venues in each geological location.

In [27]:
num_top_venues = 10

for hood in baltimore_grouped['Location']:
    print("----"+hood+"----")
    temp = baltimore_grouped[baltimore_grouped['Location'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central----
                 venue  freq
0       Sandwich Place  0.06
1                Hotel  0.04
2          Coffee Shop  0.04
3              Theater  0.03
4  American Restaurant  0.03
5    Convenience Store  0.03
6               Bakery  0.02
7           Food Truck  0.02
8                 Café  0.02
9    Indian Restaurant  0.02


----East----
                  venue  freq
0   American Restaurant  0.05
1              Bus Stop  0.05
2        Sandwich Place  0.04
3           Coffee Shop  0.04
4     Convenience Store  0.04
5                  Park  0.03
6          Intersection  0.03
7  Fast Food Restaurant  0.03
8               Theater  0.03
9    Chinese Restaurant  0.03


----North----
                 venue  freq
0          Coffee Shop  0.04
1          Pizza Place  0.04
2                  Bar  0.04
3             Pharmacy  0.03
4  American Restaurant  0.03
5       Sandwich Place  0.03
6    Convenience Store  0.02
7                 Café  0.02
8    Korean Restaurant  0.02
9         Liqu

#### Adding the data to a pandas dataframe

In [28]:
# Function sorts the values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
# Identifies the top 10 venues for each neighborhood and outputs them in a df
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Location'] = baltimore_grouped['Location']

for ind in np.arange(baltimore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(baltimore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central,Sandwich Place,Coffee Shop,Hotel,Theater,American Restaurant,Convenience Store,Café,Bakery,Bar,Pizza Place
1,East,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
2,North,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
3,Northeast,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
4,Northwest,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
5,South,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
6,Southeast,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
7,Southwest,Sandwich Place,History Museum,Discount Store,Convenience Store,Pizza Place,Coffee Shop,Grocery Store,Deli / Bodega,American Restaurant,Fried Chicken Joint
8,West,Convenience Store,American Restaurant,Grocery Store,Park,Sandwich Place,Fried Chicken Joint,Bus Stop,Bank,Chinese Restaurant,Bakery


## Clustering the neighborhoods

Import the necessary packages for this section

In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [31]:
# set number of clusters
kclusters = 5

baltimore_grouped_clustering = baltimore_grouped.drop('Location', 1)
baltimore_grouped_clustering.head(10)

,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.000000,0.007926,0.002642,0.00000,0.027741,0.000000,0.001321,0.010568,0.00000,...,0.007926,0.000000,0.000000,0.000000,0.00000,0.007926,0.000000,0.000000,0.000000,0.001321
1,0.000000,0.000000,0.000000,0.000000,0.00000,0.051136,0.000000,0.000000,0.000000,0.00000,...,0.005682,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.017045,0.000000,0.000000
2,0.005236,0.001047,0.001047,0.002094,0.00000,0.034555,0.000000,0.003141,0.000000,0.00000,...,0.002094,0.000000,0.001047,0.001047,0.00000,0.004188,0.004188,0.000000,0.001047,0.008377
3,0.004184,0.000000,0.000000,0.000000,0.00000,0.037657,0.004184,0.000000,0.016736,0.00000,...,0.000000,0.000000,0.000000,0.004184,0.00000,0.000000,0.004184,0.000000,0.008368,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.00000,0.048544,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.029126,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.00000,0.011905,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.005952,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.008929
6,0.001230,0.000000,0.001230,0.000000,0.00123,0.034440,0.000000,0.000000,0.000000,0.00492,...,0.002460,0.000000,0.000000,0.000000,0.00492,0.002460,0.003690,0.000000,0.008610,0.008610
7,0.000000,0.000000,0.003257,0.003257,0.00000,0.022801,0.000000,0.000000,0.000000,0.00000,...,0.006515,0.000000,0.003257,0.000000,0.00000,0.003257,0.000000,0.013029,0.000000,0.000000
8,0.011050,0.000000,0.005525,0.005525,0.00000,0.049724,0.000000,0.000000,0.000000,0.00000,...,0.011050,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.022099,0.000000,0.000000


Here we are running the k-means clustering and using a random state so the results can be replicated

In [32]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(baltimore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 0, 4, 2, 0, 3, 1], dtype=int32)

We are going to merge the new dataframe with the one we just created in the previous section. The merged dataframe should include Postal Code, Borough, Neighborhood, Lat, Long, the Clustering label just identified, and the top 7 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [34]:
baltimore_merged = df_final

# merge baltimore_grouped with df_final to add latitude/longitude for each neighborhood
baltimore_merged = baltimore_merged.join(neighborhoods_venues_sorted.set_index('Location'), on='Location')

In [126]:
baltimore_merged.head()

,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,North,39.326139,-76.611737,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
1,Allendale,West,39.291687,-76.676942,1,Convenience Store,American Restaurant,Grocery Store,Park,Sandwich Place,Fried Chicken Joint,Bus Stop,Bank,Chinese Restaurant,Bakery
2,Arcadia,Northeast,39.334056,-76.566500,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
3,Arlington,Northwest,39.348692,-76.682666,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
4,Armistead Gardens,Northeast,39.306646,-76.556279,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel


We will now put all the values on a map of Baltimore

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(baltimore_merged['Latitude'], baltimore_merged['Longitude'], baltimore_merged['Neighborhood'], baltimore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the clusters created

First cluster: Largest Cluster- Most Diverse

In [37]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 0, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Arcadia,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
4,Armistead Gardens,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
7,Baltimore Highlands,0,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
10,Bayview,0,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
12,Belair-Edison,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
13,Belair-Parkside,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
15,Beverly Hills,0,Italian Restaurant,Convenience Store,American Restaurant,Seafood Restaurant,Fried Chicken Joint,Discount Store,Chinese Restaurant,Pizza Place,Bar,Hotel
18,Brewer's Hill,0,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
22,Butchers Hill,0,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store
25,Canton,0,Italian Restaurant,Bar,Seafood Restaurant,American Restaurant,Mexican Restaurant,Pizza Place,Hotel,Coffee Shop,Greek Restaurant,Clothing Store


Second cluster: Park/Drinking

In [38]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 1, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Allendale,1,Convenience Store,American Restaurant,Grocery Store,Park,Sandwich Place,Fried Chicken Joint,Bus Stop,Bank,Chinese Restaurant,Bakery
8,Barclay,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
14,Berea,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
16,Biddle Street,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
19,Broadway East,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
39,Coppin Heights,1,Convenience Store,American Restaurant,Grocery Store,Park,Sandwich Place,Fried Chicken Joint,Bus Stop,Bank,Chinese Restaurant,Bakery
43,Darley Park,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
48,Eager Park,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
50,East Baltimore Midway,1,American Restaurant,Bus Stop,Convenience Store,Coffee Shop,Sandwich Place,Theater,Park,Intersection,Chinese Restaurant,Fast Food Restaurant
51,Easterwood,1,Convenience Store,American Restaurant,Grocery Store,Park,Sandwich Place,Fried Chicken Joint,Bus Stop,Bank,Chinese Restaurant,Bakery


Third cluster: Social groups

In [39]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 2, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Arundel Cove,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
20,Brooklyn,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
21,Brooklyn Homes,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
35,Cherry Hill,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
41,Curtis Bay,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
60,Federal Hill,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
84,Harborview,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
87,Hawkins Point,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
109,Lakeland,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store
117,Locust Point,2,Bar,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Gym,Liquor Store


Fourth Cluster: Restaurants

In [40]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 3, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
9,Barre Circle,3,Sandwich Place,History Museum,Discount Store,Convenience Store,Pizza Place,Coffee Shop,Grocery Store,Deli / Bodega,American Restaurant,Fried Chicken Joint
11,Beechfield,3,Sandwich Place,History Museum,Discount Store,Convenience Store,Pizza Place,Coffee Shop,Grocery Store,Deli / Bodega,American Restaurant,Fried Chicken Joint
17,Bolton Hill,3,Sandwich Place,Coffee Shop,Hotel,Theater,American Restaurant,Convenience Store,Café,Bakery,Bar,Pizza Place
24,Cameron Village,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
...,...,...,...,...,...,...,...,...,...,...,...,...
213,Woodbourne Heights,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
214,Woodbourne-McCabe,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
217,Wyman Park,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park
218,Wyndhurst,3,Coffee Shop,Pizza Place,Bar,American Restaurant,Pharmacy,Sandwich Place,Café,Chinese Restaurant,Convenience Store,Park


Fifth Cluster: Quick bites 

In [41]:
baltimore_merged.loc[baltimore_merged['Cluster Labels'] == 4, baltimore_merged.columns[[0] + list(range(4, baltimore_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Arlington,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
6,Ashburton,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
23,Callaway-Garrison,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
31,Central Park Heights,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
36,Cheswolde,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
40,Cross Country,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
45,Dolfield,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
46,Dorchester,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
49,East Arlington,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint
59,Fallstaff,4,Liquor Store,American Restaurant,Deli / Bodega,Discount Store,Chinese Restaurant,Arts & Crafts Store,Pharmacy,Park,Home Service,Wings Joint


### Examination by geological location
After using k-means on the geological location, the clusters were grouped similar to the location. We will now look complete the same analysis, but this time we will be using the neighborhood names instead to see if the grouping is similar or different.

### Start of Neighborhood Analysis

In [42]:
def getNearbyVenues2(names, latitudes, longitudes, radius=500):
    
    venues_list2=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list2.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues2 = pd.DataFrame([item for venue_list2 in venues_list2 for item in venue_list2])
    nearby_venues2.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues2)

In [43]:
baltimore2_venues = getNearbyVenues2(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

Abell
Allendale
Arcadia
Arlington
Armistead Gardens
Arundel Cove
Ashburton
Baltimore Highlands
Barclay
Barre Circle
Bayview
Beechfield
Belair-Edison
Belair-Parkside
Berea
Beverly Hills
Biddle Street
Bolton Hill
Brewer's Hill
Broadway East
Brooklyn
Brooklyn Homes
Butchers Hill
Callaway-Garrison
Cameron Village
Canton
Carrollton Ridge
Cathedral Hill
Cedarcroft
Cedmont
Cedonia
Central Park Heights
Charles Center
Charles North
Charles Village
Cherry Hill
Cheswolde
Christopher
Cold Spring
Coppin Heights
Cross Country
Curtis Bay
Cylburn
Darley Park
Dickeyville
Dolfield
Dorchester
Druid Heights
Eager Park
East Arlington
East Baltimore Midway
Easterwood
Eastwood
Edmondson
Edmondson Village
Ednor Gardens-Lakeside
Ellwood Park
Evergreen
Fairmount
Fallstaff
Federal Hill
Fells Point
Fells Prospect
Forest Park
Frankford
Franklin Square
Franklintown
Gardenville
Gay Street
Glen
Glen Oaks
Glenham-Belford
Govans
Graceland Park
Greektown
Greenmount West
Greenspring
Grove Park
Guilford
Gwynn Oak
Hampden


In [44]:
print(baltimore2_venues.shape)
baltimore2_venues.head()

(3867, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abell,39.326139,-76.611737,32nd Street Farmer's Market,39.327362,-76.610851,Farmers Market
1,Abell,39.326139,-76.611737,Pete's Grille,39.327064,-76.609593,Breakfast Spot
2,Abell,39.326139,-76.611737,The Book Thing,39.325253,-76.610272,Bookstore
3,Abell,39.326139,-76.611737,Normal's Books & Records,39.326012,-76.609903,Record Shop
4,Abell,39.326139,-76.611737,Peabody Heights Brewery,39.324744,-76.610819,Brewery


In [45]:
print(baltimore2_venues.shape)
baltimore2_venues.head()

(3867, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abell,39.326139,-76.611737,32nd Street Farmer's Market,39.327362,-76.610851,Farmers Market
1,Abell,39.326139,-76.611737,Pete's Grille,39.327064,-76.609593,Breakfast Spot
2,Abell,39.326139,-76.611737,The Book Thing,39.325253,-76.610272,Bookstore
3,Abell,39.326139,-76.611737,Normal's Books & Records,39.326012,-76.609903,Record Shop
4,Abell,39.326139,-76.611737,Peabody Heights Brewery,39.324744,-76.610819,Brewery


In [47]:
# one hot encoding
baltimore2_onehot = pd.get_dummies(baltimore2_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
baltimore2_onehot['Neighborhood'] = baltimore2_venues['Neighborhood']

In [48]:
# move neighborhood column to the first column
neighborhood = baltimore2_onehot['Neighborhood']
baltimore2_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
baltimore2_onehot.insert(0, 'Neighborhood', neighborhood)
baltimore2_onehot

,Neighborhood,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3862,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3863,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3864,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3865,York Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
baltimore2_onehot.shape

(3867, 286)

In [50]:
baltimore2_grouped = baltimore2_onehot.groupby('Neighborhood').mean().reset_index()
baltimore2_grouped

,Neighborhood,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abell,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.00000
1,Allendale,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.00000
2,Arcadia,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.00000
3,Arlington,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.1,0.000000,0.00000
4,Armistead Gardens,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Woodring,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.00000
209,Wyman Park,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.02381,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,0.0,0.000000,0.02381
210,Wyndhurst,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.090909,0.00000
211,Yale Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.00000


In [51]:
num_top_venues = 10

for hood in baltimore2_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = baltimore2_grouped[baltimore2_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abell----
                venue  freq
0         Pizza Place  0.08
1  Chinese Restaurant  0.08
2      Sandwich Place  0.06
3            Pharmacy  0.06
4           Bookstore  0.04
5       Grocery Store  0.04
6                 Bar  0.04
7                Café  0.04
8   Convenience Store  0.04
9         Coffee Shop  0.04


----Allendale----
                   venue  freq
0                    Bar  0.33
1           Home Service  0.33
2    American Restaurant  0.33
3                    ATM  0.00
4  Performing Arts Venue  0.00
5               Pharmacy  0.00
6              Pet Store  0.00
7            Pet Service  0.00
8    Peruvian Restaurant  0.00
9     Persian Restaurant  0.00


----Arcadia----
                   venue  freq
0     Chinese Restaurant   0.5
1      Other Repair Shop   0.5
2  Performing Arts Venue   0.0
3              Piano Bar   0.0
4               Pharmacy   0.0
5              Pet Store   0.0
6            Pet Service   0.0
7    Peruvian Restaurant   0.0
8     Persian Restau

In [52]:
# Function sorts the values in descending order
def return_most_common_venues2(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
# Identifies the top 10 venues for each neighborhood and outputs them in a df
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted2 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted2['Neighborhood'] = baltimore2_grouped['Neighborhood']

for ind in np.arange(baltimore2_grouped.shape[0]):
    neighborhoods_venues_sorted2.iloc[ind, 1:] = return_most_common_venues2(baltimore2_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,Chinese Restaurant,Pizza Place,Pharmacy,Sandwich Place,Bookstore,Bar,Convenience Store,Coffee Shop,Café,Grocery Store
1,Allendale,Bar,American Restaurant,Home Service,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market
2,Arcadia,Other Repair Shop,Chinese Restaurant,Food Truck,Food Stand,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Exhibit
3,Arlington,Caribbean Restaurant,Southern / Soul Food Restaurant,Discount Store,Seafood Restaurant,Pharmacy,Bakery,Check Cashing Service,Convenience Store,Wings Joint,Food
4,Armistead Gardens,Seafood Restaurant,Rental Service,Convenience Store,Bar,Sandwich Place,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market
...,...,...,...,...,...,...,...,...,...,...,...
208,Woodring,German Restaurant,Baseball Field,Frame Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Yoga Studio
209,Wyman Park,Pizza Place,Bakery,Café,Spa,Pet Store,Coffee Shop,Seafood Restaurant,Bike Shop,Bookstore,Movie Theater
210,Wyndhurst,Bank,Grocery Store,Clothing Store,Women's Store,Spa,Pharmacy,Flower Shop,Furniture / Home Store,Toy / Game Store,Coffee Shop
211,Yale Heights,Laundromat,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Yoga Studio


# Clustering Baltimore Neighborhoods

In [56]:
# set number of clusters
kclusters = 5

baltimore2_grouped_clustering = baltimore2_grouped.drop('Neighborhood', 1)
baltimore2_grouped_clustering.head(10)

,ATM,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.00,0.150000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.00,0.064516,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(baltimore2_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 3, 3, 3, 3, 0, 3, 3], dtype=int32)

In [93]:
# add clustering labels
neighborhoods_venues_sorted2.insert(0, 'Cluster Labels', kmeans.labels_)

AttributeError: 'NoneType' object has no attribute 'astype'

In [104]:
baltimore2_merged = df_final

# merge baltimore_grouped with df_final to add latitude/longitude for each neighborhood
baltimore2_merged = baltimore2_merged.join(neighborhoods_venues_sorted2.set_index('Neighborhood'), on='Neighborhood')

del baltimore2_merged['Cluster Labels2']
baltimore2_merged.head()


,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,North,39.326139,-76.611737,3.0,Chinese Restaurant,Pizza Place,Pharmacy,Sandwich Place,Bookstore,Bar,Convenience Store,Coffee Shop,Café,Grocery Store
1,Allendale,West,39.291687,-76.676942,0.0,Bar,American Restaurant,Home Service,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market
2,Arcadia,Northeast,39.334056,-76.566500,0.0,Other Repair Shop,Chinese Restaurant,Food Truck,Food Stand,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Exhibit
3,Arlington,Northwest,39.348692,-76.682666,3.0,Caribbean Restaurant,Southern / Soul Food Restaurant,Discount Store,Seafood Restaurant,Pharmacy,Bakery,Check Cashing Service,Convenience Store,Wings Joint,Food
4,Armistead Gardens,Northeast,39.306646,-76.556279,3.0,Seafood Restaurant,Rental Service,Convenience Store,Bar,Sandwich Place,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market


In [123]:

baltimore2_merged['Cluster Labels'].replace(np.nan, 3, inplace=True)
baltimore2_merged['Cluster Labels'] = baltimore2_merged['Cluster Labels'].astype(int)
baltimore2_merged.dtypes

Neighborhood               object
Location                   object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [64]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 0, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Allendale,0.0,Bar,American Restaurant,Home Service,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market
2,Arcadia,0.0,Other Repair Shop,Chinese Restaurant,Food Truck,Food Stand,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Exhibit
7,Baltimore Highlands,0.0,Bar,Airport,Convenience Store,Light Rail Station,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Field,Fish Market
12,Belair-Edison,0.0,Fried Chicken Joint,Chinese Restaurant,ATM,American Restaurant,Discount Store,Pharmacy,Convenience Store,Food Court,Food & Drink Shop,Food Service
19,Broadway East,0.0,Bar,Breakfast Spot,Park,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flower Shop
33,Charles North,0.0,Bar,Korean Restaurant,Theater,American Restaurant,Art Gallery,Music Venue,Gay Bar,Rock Club,Tapas Restaurant,Coffee Shop
73,Graceland Park,0.0,Pharmacy,Bar,Frame Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market
75,Greenmount West,0.0,Theater,Bar,Art Gallery,Nightclub,Bus Stop,Chinese Restaurant,Tapas Restaurant,Sandwich Place,Coffee Shop,Train Station
77,Grove Park,0.0,Clothing Store,Liquor Store,American Restaurant,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market
111,Lauraville,0.0,Bar,Discount Store,Coffee Shop,Bus Stop,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Fish Market


In [65]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 1, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
165,Purnell,1.0,Construction & Landscaping,Yoga Studio,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flower Shop


In [66]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 2, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Callaway-Garrison,2.0,Grocery Store,Yoga Studio,Frame Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market
88,Heritage Crossing,2.0,Grocery Store,Yoga Studio,Frame Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market


In [67]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 3, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abell,3.0,Chinese Restaurant,Pizza Place,Pharmacy,Sandwich Place,Bookstore,Bar,Convenience Store,Coffee Shop,Café,Grocery Store
3,Arlington,3.0,Caribbean Restaurant,Southern / Soul Food Restaurant,Discount Store,Seafood Restaurant,Pharmacy,Bakery,Check Cashing Service,Convenience Store,Wings Joint,Food
4,Armistead Gardens,3.0,Seafood Restaurant,Rental Service,Convenience Store,Bar,Sandwich Place,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market
5,Arundel Cove,3.0,Harbor / Marina,Gym,Convenience Store,Food Court,Baseball Field,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
6,Ashburton,3.0,Home Service,Lake,Arts & Crafts Store,Yoga Studio,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
...,...,...,...,...,...,...,...,...,...,...,...,...
216,Woodring,3.0,German Restaurant,Baseball Field,Frame Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Yoga Studio
217,Wyman Park,3.0,Pizza Place,Bakery,Café,Spa,Pet Store,Coffee Shop,Seafood Restaurant,Bike Shop,Bookstore,Movie Theater
218,Wyndhurst,3.0,Bank,Grocery Store,Clothing Store,Women's Store,Spa,Pharmacy,Flower Shop,Furniture / Home Store,Toy / Game Store,Coffee Shop
219,Yale Heights,3.0,Laundromat,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Yoga Studio


In [68]:
baltimore2_merged.loc[baltimore2_merged['Cluster Labels'] == 4, baltimore2_merged.columns[[0] + list(range(4, baltimore2_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Cheswolde,4.0,Home Service,Yoga Studio,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market
87,Hawkins Point,4.0,Business Service,Home Service,Yoga Studio,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market
146,Orchard Ridge,4.0,Home Service,Brewery,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flower Shop
204,West Hills,4.0,Home Service,Park,Yoga Studio,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market
